In [1]:
# %load /home/biswadip/autoreload_jn.py
%load_ext autoreload
%autoreload 2


In [2]:
from collections import defaultdict, Counter
from tqdm import tqdm_notebook
import re

In [49]:
import numpy as np
import pandas as pd
%matplotlib inline

In [53]:
from models.text import CustomTokenizer
from models.model import word_list, max_sentences, maxlen

In [54]:
word_list[0]

'dummy_word'

In [7]:
# word_vectors - word_vector_dict["0"]

In [8]:
tokenizer = CustomTokenizer(word_list = word_list)

In [9]:
tokenizer.doc_to_sequences(["unk hello. hello guy", "good wah"])

[[[399999, 13075], [13075, 1856]], [[219, 11577]]]

### Loading reviews

In [10]:
import os

In [11]:
path = "negativeReviews/"
neg_reviews = []
for f in os.listdir(path):
    file = os.path.join(path, f)
    with open(file, "r") as fl:
        neg_reviews.append(fl.read())
    

In [12]:
# neg_reviews[0]

In [13]:
path = "positiveReviews//"
pos_reviews = []
for f in os.listdir(path):
    file = os.path.join(path, f)
    with open(file, "r") as fl:
        pos_reviews.append(fl.read())
    

In [14]:
# pos_reviews[0]

In [15]:
data = pd.DataFrame({"text":neg_reviews, "sentiment":0}).append(pd.DataFrame({"text":pos_reviews, "sentiment":1}))

In [16]:
data.to_csv("tagged_data.csv")

In [17]:
data = data.reset_index()

In [18]:
data = data.filter(["text","sentiment"])

In [81]:
# data

### Mapping sentences to sequence of word vectors

In [19]:
import tensorflow as tf

In [20]:
track = []
for doc in data.text.values:
    sents = doc.split(".")
    for sent in sents:
        track.append(len(sent.split()))
    

In [21]:
# pd.Series(track).hist(range=(0,100))

In [68]:
inp = tokenizer.doc_to_sequences(data.text.tolist()[0:1000])

inputs = []
for doc in inp:
    inputs.append(
        tf.keras.preprocessing.sequence.pad_sequences(
            doc, padding="post", value=0, maxlen=maxlen, dtype=None
        )
    )

In [69]:
a = np.zeros((len(inputs),max_sentences,maxlen))

In [70]:
for row,x in zip(a, inputs):
    row[:len(x)] = x[:50] 

In [71]:
a.shape

(1000, 50, 80)

### Define Model

In [77]:
from models.model import model
from models.data import Sequence_generator

In [73]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])

In [75]:
# model(a)

In [29]:
units = 20

In [474]:
embedding_len = 50

In [215]:
max_sentences = 15
maxlen = 100

In [84]:
# pd.get_dummies(data.sentiment)

In [ ]:
model.fit(pd.get_dummies(trainData.sentiment).values)

In [78]:
batch_s = 5

In [85]:
hs = model.fit_generator(
        Sequence_generator(
            a,pd.get_dummies(data.sentiment).values[:1000],
            batch_s
        ), 
        steps_per_epoch=int(1000/batch_s), 
        epochs=2, 
        verbose=2
    )

Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 200 steps
Epoch 1/2


KeyboardInterrupt: 